# Submit nudget elastic band or replica chain calculation

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# General imports.
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA imports.
%load_ext aiida
%aiida

# AiiDAlab imports.
import aiidalab_widgets_base as awb
from aiida import orm, plugins

from surfaces_tools.utils import wfn

# Custom imports.
from surfaces_tools.widgets import build_slab, cdxml, computational_resources, inputs

In [ ]:
Cp2kNebWorkChain = plugins.WorkflowFactory("nanotech_empa.cp2k.neb")
Cp2kReplicaWorkChain = plugins.WorkflowFactory("nanotech_empa.cp2k.replica")

In [ ]:
# Structure selector.
build_slab = build_slab.BuildSlab(title="Build slab")
input_details = inputs.InputDetails()


structure_selector = awb.StructureManagerWidget(
    importers=[
        awb.StructureUploadWidget(title="Import from computer"),
        awb.StructureBrowserWidget(title="AiiDA database"),
        awb.SmilesWidget(title="From SMILES"),
        cdxml.CdxmlUpload2GnrWidget(title="CDXML"),
    ],
    editors=[awb.BasicStructureEditor(title="Edit structure"), build_slab],
    storable=False,
    node_class="StructureData",
)

ipw.dlink((structure_selector, "structure"), (build_slab, "molecule"))
ipw.dlink((structure_selector, "structure"), (input_details, "structure"))
ipw.dlink((input_details, "details"), (build_slab, "details"))


display(structure_selector)

replica_or_neb = ipw.ToggleButtons(
    options=["Replica", "NEB"],
    description="Type:",
    disabled=False,
    button_style="",
    tooltips=["Replica", "NEB"],
    icons=["check", "check"],
    style={"description_width": "initial"},
    value="NEB",
)

# Code.
code_input_widget = awb.ComputationalResourcesWidget(
    description="CP2K code:", default_calc_job_plugin="cp2k"
)
resources = computational_resources.ProcessResourcesWidget()

# Global variables.
label = "CP2K_NEB"
the_workchain = Cp2kNebWorkChain

# Protocol.
protocol = ipw.Dropdown(
    value="standard",
    options=[
        ("Standard", "standard"),
        ("Low accuracy", "low_accuracy"),
        ("Debug", "debug"),
    ],
    description="Protocol:",
    style={"description_width": "initial"},
)

output = ipw.Output()

In [ ]:
workflow_description = ipw.Text(
    description="Workflow description:",
    placeholder="Provide the description here.",
    style={"description_width": "initial"},
    layout={"width": "70%"},
)

In [ ]:
ipw.dlink((code_input_widget, "value"), (input_details, "selected_code"))


def prepare_inputs():
    with output:
        clear_output()
    if not structure_selector.structure_node:
        can_submit, msg = False, "Select a structure furst."
    elif not code_input_widget.value:
        can_submit, msg = False, "Select CP2K code."
    else:
        can_submit, msg, parameters = input_details.return_final_dictionary()

    if not can_submit:
        with output:
            print(msg)
            return

    builder = the_workchain.get_builder()
    builder.protocol = orm.Str(protocol.value)
    builder.metadata.label = label
    builder.metadata.description = workflow_description.value
    builder.code = orm.load_code(code_input_widget.value)
    builder.options = {
        "max_wallclock_seconds": resources.walltime_seconds,
        "resources": {
            "num_machines": resources.nodes,
            "num_mpiprocs_per_machine": resources.tasks_per_node,
            "num_cores_per_mpiproc": resources.threads_per_task,
        },
    }

    builder.structure = structure_selector.structure_node
    builder.dft_params = orm.Dict(parameters["dft_params"])
    builder.sys_params = orm.Dict(parameters["sys_params"])
    if "neb_params" in parameters:
        builder.neb_params = orm.Dict(parameters["neb_params"])

    if "replica_uuids" in parameters:
        replicas = {}
        for i in range(len(parameters["replica_uuids"])):
            name = "replica_" + str(i + 1).zfill(3)
            replicas[name] = orm.load_node(parameters["replica_uuids"][i])
        builder.replicas = replicas

    if "restart_from" in parameters:
        builder.restart_from = orm.Str(parameters["restart_from"])
    elif replica_or_neb.value == "Replica":
        # Check if a restart wfn is available for the replica chain calculations not restarting from a previous calculation.
        wave_function = None
        code = orm.load_code(code_input_widget.value)
        if structure_selector.structure_node.is_stored:
            wave_function = wfn.structure_available_wfn(
                node=structure_selector.structure_node,
                relative_replica_id=None,
                current_hostname=code.computer.hostname,
                return_path=False,
                dft_params=parameters["dft_params"],
            )
        if wave_function is not None:
            print(f"Restarting from wfn in folder: {wave_function.pk}")
            builder.parent_calc_folder = wave_function

    return builder

In [ ]:
btn_output = ipw.Output()
btn_submit_neb = awb.SubmitButtonWidget(
    Cp2kNebWorkChain,
    inputs_generator=prepare_inputs,
    disable_after_submit=False,
    append_output=True,
)
btn_submit_replica = awb.SubmitButtonWidget(
    Cp2kReplicaWorkChain,
    inputs_generator=prepare_inputs,
    disable_after_submit=False,
    append_output=True,
)

with btn_output:
    display(btn_submit_neb)


def on_replica_or_neb_change(change):
    global label, the_workchain
    with btn_output:
        clear_output()
        if change["new"] == "Replica":
            label = "CP2K_Replica"
            the_workchain = Cp2kReplicaWorkChain
            input_details.replica = True
            input_details.neb = False
            display(btn_submit_replica)
        elif change["new"] == "NEB":
            label = "CP2K_NEB"
            the_workchain = Cp2kNebWorkChain
            input_details.replica = False
            input_details.neb = True
            display(btn_submit_neb)


replica_or_neb.observe(on_replica_or_neb_change, names="value")

In [ ]:
# Resources estimation.
resources_estimation_button = computational_resources.ResourcesEstimatorWidget()
resources_estimation_button.link_to_resources_widget(resources)
ipw.dlink((structure_selector, "structure"), (input_details, "structure"))
ipw.dlink((input_details, "details"), (resources_estimation_button, "details"))
ipw.dlink((input_details, "uks"), (resources_estimation_button, "uks"))
ipw.dlink((input_details, "neb"), (resources, "neb"))
ipw.dlink((input_details, "n_replica_trait"), (resources, "n_replica_trait"))
ipw.dlink(
    (input_details, "n_replica_per_group_trait"),
    (resources, "n_replica_per_group_trait"),
)
ipw.dlink((resources, "nproc_replica_trait"), (input_details, "nproc_replica_trait"))
_ = ipw.dlink(
    (code_input_widget, "value"), (resources_estimation_button, "selected_code")
)
input_details.replica = False
input_details.neb = True

# Inputs

In [ ]:
display(protocol, replica_or_neb, input_details)

# Code and resources

In [ ]:
display(code_input_widget, resources, resources_estimation_button)

# Submit

In [ ]:
display(workflow_description, btn_output, output)